<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/QuantumCircuit_ParametrizedZGate_mkbahk_20250916.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### Qiskit 설치: Windows, 클라우드 환경 ###
%pip install qiskit[visualization]==1.2.4
%pip install qiskit-ibm-runtime==0.30.0
%pip install qiskit-transpiler-service==0.4.10
%pip install qiskit-aer==0.15.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 16.8 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=329da47c53adf2620167a4deb052226a0da70a3cfbeb13de7fd94db6997f0c01
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.8/366.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_aer import AerSimulator, StatevectorSimulator
from qiskit.visualization import visualize_transition  # 옵션: 회전 시각화

In [4]:
# 매개변수 설정
n = 2  # A의 큐비트 수 (예시)

# 양자 레지스터 생성
a_reg = QuantumRegister(n, 'a')  # A 레지스터 (양자 상태)
target_reg = QuantumRegister(1, 'target')  # 타겟 큐비트

# 양자 회로 생성
qc = QuantumCircuit(a_reg, target_reg)

# 타겟을 |1> 상태로 초기화 (위상 변화를 관찰하기 위해)
qc.x(target_reg[0])

# A 레지스터를 특정 상태로 초기화 (예: A=1, 이진 01, LSB a[0]=1, MSB a[1]=0)
# 주석 해제하여 사용; 또는 중첩을 위해 qc.h(a_reg) 등 추가 가능
qc.x(a_reg[0])  # a[0] = 1 (LSB)

# Parametrized Z Gate 구현: Controlled-RZ (CPHASE)로 분해
# A / 2^n * π = sum_{j=0}^{n-1} a[j] * (π / 2^{n-j})
for j in range(n):
    phi = np.pi / (2 ** (n - j))  # 각 비트에 대한 위상 각도
    qc.cp(phi, a_reg[j], target_reg[0])  # Controlled Phase Gate (CP)

# 배리어 추가 (시각화 목적, 옵션)
qc.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'a'), 0), Qubit(QuantumRegister(2, 'a'), 1), Qubit(QuantumRegister(1, 'target'), 0)), clbits=())

In [14]:
# 회로 출력 (옵션)
print(qc.draw())

        ┌───┐                   ░ 
   a_0: ┤ X ├─■─────────────────░─
        └───┘ │                 ░ 
   a_1: ──────┼────────■────────░─
        ┌───┐ │P(π/4)  │P(π/2)  ░ 
target: ┤ X ├─■────────■────────░─
        └───┘                   ░ 


In [16]:
# 시뮬레이션
simulator = StatevectorSimulator()
result = simulator.run(qc).result()
statevector = result.get_statevector(qc)

In [17]:
# 결과 출력
print("최종 상태 벡터:", statevector)

최종 상태 벡터: Statevector([0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.        +0.j        ,
             0.        +0.j        , 0.70710678+0.70710678j,
             0.        +0.j        , 0.        +0.j        ],
            dims=(2, 2, 2))
